## MNIST Fashion Dataset Maker
### Client NonIID

In [22]:
import os
import copy
import random
import shutil

import tensorflow as tf
import numpy as np
import pandas as pd
import PIL.Image as Image

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
images = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))

In [4]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data
labelnames = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
labelnames

['T-shirt',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [ ]:
div = pd.read_csv('div.csv')
div.fillna(0, inplace=True)
div

In [ ]:
# Count each labels
total = {}
for idx, data in enumerate(zip(images, labels), start=0):
    label = labelnames[data[1]]
    total[label] = total.get(label, 0) + 1
total

In [ ]:
for column in div.columns[1:]:
    div = div.assign(**{column: np.around(div.loc[:,column]*total[column])})
div

In [ ]:
counter = {}
output = os.path.abspath(os.path.expanduser('dataset-mnistfashion'))
for idx, data in enumerate(zip(images, labels), start=0):
    image = Image.fromarray(data[0])
    label = labelnames[data[1]]
    num = counter.get(label, 0)
    party = random.choice(div.loc[div.loc[:, label] > 0, 'Clients'].values[:-1])
    odir = os.path.join(output, f'{party}', label)
    os.makedirs(odir, exist_ok=True)
    opath = os.path.join(odir, f'{num:04d}.jpg')
    image.save(opath)
    div.loc[div.loc[:, 'Clients'] == party, label] = div.loc[div.loc[:, 'Clients'] == party, label] - 1
    counter[label] = num + 1

In [ ]:
div

### split test dataset for each clients

In [5]:
div = pd.read_csv('div.csv')
div.fillna(0, inplace=True)
div

,Clients,T-shirt,Trouser,Pullover,Dress,Coat,Sandal,Shirt,Sneaker,Bag,Ankle boot
0,c1,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.3
1,c2,0.3,0.3,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0
2,c3,0.0,0.0,0.3,0.3,0.5,0.5,0.0,0.0,0.0,0.0
3,c4,0.0,0.0,0.0,0.0,0.3,0.3,0.5,0.5,0.0,0.0
4,c5,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.3,0.5,0.5
5,test,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
6,sum,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
pathlist = [os.path.abspath(os.path.expanduser('dataset-mnistfashion/test'))]

In [17]:
pathqueue = copy.deepcopy(pathlist)
total = {}
while len(pathqueue) > 0:
    cpath = pathqueue.pop()
    with os.scandir(cpath) as it:
        for entry in it:
            if not entry.name.startswith('.') and entry.is_dir():
                pathqueue.append(entry.path)
            elif not entry.name.startswith('.') and entry.is_file():
                label = entry.path.split('/')[-2]
                total[label] = total.get(label, 0) + 1
total

{'T-shirt': 1400,
 'Pullover': 1400,
 'Shirt': 1400,
 'Coat': 1400,
 'Sandal': 1400,
 'Bag': 1400,
 'Trouser': 1400,
 'Sneaker': 1400,
 'Ankle boot': 1400,
 'Dress': 1400}

In [20]:
for column in div.columns[1:]:
    div = div.assign(**{column: np.around(div.loc[:,column]*total[column])})
div

,Clients,T-shirt,Trouser,Pullover,Dress,Coat,Sandal,Shirt,Sneaker,Bag,Ankle boot
0,c1,700.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,420.0,420.0
1,c2,420.0,420.0,700.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0
2,c3,0.0,0.0,420.0,420.0,700.0,700.0,0.0,0.0,0.0,0.0
3,c4,0.0,0.0,0.0,0.0,420.0,420.0,700.0,700.0,0.0,0.0
4,c5,0.0,0.0,0.0,0.0,0.0,0.0,420.0,420.0,700.0,700.0
5,test,280.0,280.0,280.0,280.0,280.0,280.0,280.0,280.0,280.0,280.0
6,sum,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0


In [23]:
counter = {}
output = os.path.abspath(os.path.expanduser('testset-mnistfashion'))

pathqueue = copy.deepcopy(pathlist)
total = {}
while len(pathqueue) > 0:
    cpath = pathqueue.pop()
    with os.scandir(cpath) as it:
        for entry in it:
            if not entry.name.startswith('.') and entry.is_dir():
                pathqueue.append(entry.path)
            elif not entry.name.startswith('.') and entry.is_file():
                label = entry.path.split('/')[-2]
                party = random.choice(div.loc[div.loc[:, label] > 0, 'Clients'].values[:-1])
                odir = os.path.join(output, f'{party}', label)
                os.makedirs(odir, exist_ok=True)
                shutil.copy(entry.path, odir)
                div.loc[div.loc[:, 'Clients'] == party, label] = div.loc[div.loc[:, 'Clients'] == party, label] - 1
                total[label] = total.get(label, 0) + 1

In [24]:
div

,Clients,T-shirt,Trouser,Pullover,Dress,Coat,Sandal,Shirt,Sneaker,Bag,Ankle boot
0,c1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,c2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,c3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,c5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,test,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,sum,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0
